In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [13]:
test_data = pd.read_csv('test.csv')

train_data = pd.read_csv('train.csv')

In [14]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [15]:
train_data = train_data.drop(['Name', 'Embarked', 'Ticket', 'Cabin'], axis=1)

label_encoder = LabelEncoder()
train_data['Sex'] = label_encoder.fit_transform(train_data['Sex'])

In [18]:
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,1,22.0,1,0,7.2500
1,2,1,1,0,38.0,1,0,71.2833
2,3,1,3,0,26.0,0,0,7.9250
3,4,1,1,0,35.0,1,0,53.1000
4,5,0,3,1,35.0,0,0,8.0500


In [16]:
df_all_corr = train_data.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()

# Rename columns for better readability
df_all_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)

# Filter the results for correlations involving the 'Age' feature
age_corr = df_all_corr[df_all_corr['Feature 1'] == 'Age']

# Display the filtered correlations
print(age_corr)

   Feature 1    Feature 2  Correlation Coefficient
4        Age          Age                 1.000000
14       Age       Pclass                 0.369226
18       Age        SibSp                 0.308247
26       Age        Parch                 0.189119
36       Age         Fare                 0.096067
38       Age          Sex                 0.093254
44       Age     Survived                 0.077221
51       Age  PassengerId                 0.036847


In [19]:
# Calculate the median age for each group (Sex and Pclass)
age_by_pclass_sex = train_data.groupby(['Sex', 'Pclass']).median()['Age']

# Display the median ages for each group
for pclass in range(1, 4):
    for sex in [0, 1]:
        print('Median age of Pclass {} {}s: {}'.format(pclass, sex, age_by_pclass_sex[sex][pclass]))

# Calculate and print the overall median age
print('Median age of all passengers: {}'.format(train_data['Age'].median()))

# Fill missing Age values with the medians of Sex and Pclass groups
train_data['Age'] = train_data.groupby(['Sex', 'Pclass'])['Age'].apply(lambda x: x.fillna(x.median()))


Median age of Pclass 1 0s: 35.0
Median age of Pclass 1 1s: 40.0
Median age of Pclass 2 0s: 28.0
Median age of Pclass 2 1s: 30.0
Median age of Pclass 3 0s: 21.5
Median age of Pclass 3 1s: 25.0
Median age of all passengers: 28.0


TypeError: incompatible index of inserted column with frame index